In [1]:
import numpy as np
import os
import pandas as pd
from tqdm.notebook import tqdm
import copy
import sys
# sys.setrecursionlimit(10000)

In [2]:
class Graph:
    
    def __init__(self, n):
        self.n_nodes = n
        self.matrix = np.zeros((n, n), dtype="uint8")
        self.nodes = [set() for i in range(n)]
#         self.nodes = np.full(n, fill_value=set(), dtype="object")
    
    
    def add_edge(self, v, w):
        
        if v == w:
            self.matrix[v-1, v-1] = 2
            self.nodes[v-1].add(v)
            
        else:
            self.matrix[v-1, w-1] = 1
            self.matrix[w-1, v-1] = 1
            self.nodes[v-1].add(w)
            self.nodes[w-1].add(v)
    
    def get_node(self, v):
        return self.nodes[v-1]
    
    def get_lists(self):
        return self.nodes
    
    def get_node_edges(self):
        return {i+1:self.nodes[i] for i in range(self.n_nodes)}
    
    def get_matrix(self):
        return self.matrix
    
    def get_matrix_beautiful(self):
        return pd.DataFrame(self.matrix, columns=np.arange(1, self.n_nodes+1), index=np.arange(1, self.n_nodes+1))
    
    def sort_neighbors(self):
        self.nodes = [sorted(n) for n in self.nodes]

In [3]:
def open_graph_txt(filename, extra=False):
    with open(filename, "r") as f:
        lines = f.read().split("\n")
        n_nodes = int(lines[0])
        edges = [tuple(map(lambda i: int(i), line.split(" "))) for line in lines[1:-1]]
    
    graph = Graph(n_nodes)
    for v, w in edges:
        graph.add_edge(v, w)
    
    if extra:
        return graph, n_nodes, edges
    
    return graph

In [4]:
folder = "inputs"
filename = "grafo_4.txt"

path = os.path.join(folder, filename)

In [5]:
graph = open_graph_txt(path)
graph.sort_neighbors()

In [ ]:
graph.nodes

In [ ]:
graph.get_node_edges()

In [ ]:
graph.get_lists()

In [ ]:
graph.get_matrix()

In [ ]:
graph.get_matrix_beautiful()

In [ ]:
def graph_statistics(graph):
    print("Número de vértices:", graph.n_nodes)
    print("Número de arestas:", graph.get_matrix().sum()/2)
    print("Grau mínimo:", graph.get_matrix().sum(axis=0).min())
    print("Grau máximo:", graph.get_matrix().sum(axis=0).max())
    print("Grau médio:", graph.get_matrix().sum(axis=0).mean())
    print("Mediana do Grau:", np.median(graph.get_matrix().sum(axis=0)))

In [ ]:
graph_statistics(graph)

In [ ]:
class DFS:
    def __init__(self, graph, root):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.level = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        
        self.level[root-1] = 0
#         self.visited[root-1] = 1
        
        self.start_root(root)
    
    def start_root(self, root):
        self.stack = []
        self.stack.append(root)
    
    def search(self):
        
        while(len(self.stack) != 0):
            print(self.stack)
            u = self.stack.pop()
            if (not self.visited[u-1]):
                self.visited[u-1] = 1
                
                for v in self.graph.nodes[u-1]:
                    if v == u:
                        continue
                        
                    self.stack.append(v)
                    if not self.visited[v-1]:
                        self.level[v-1] = self.level[u-1] + 1
                        self.parent[v-1] = u

In [ ]:
dfs = DFS(graph, 1)
dfs.search()

In [ ]:
[1, 2, 5, 3]

In [ ]:
pd.DataFrame(list(zip(range(1, dfs.graph.n_nodes+1), dfs.level, dfs.parent)), columns=["node", "level", "parent"], index=np.arange(1, dfs.graph.n_nodes+1))

In [ ]:
net.show("graph.html")

In [8]:
class BFS:
    def __init__(self, graph, root):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.level = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        self.parent = np.full(graph.n_nodes, fill_value=-1, dtype="int32")
        
        self.level[root-1] = 0
        self.visited[root-1] = 1
        
        self.start_root(root)
        
    def start_root(self, root):
        self.queue = []
        self.queue.append(root)
        
    def search(self):
        
        while(len(self.queue)):
            v = self.queue.pop(0)
            
            for w in self.graph.nodes[v-1]:
                if v == w:
                        continue
                if not self.visited[w-1]:
                    self.visited[w-1] = 1
                    self.queue.append(w)
                    self.parent[w-1] = v
                    self.level[w-1] = self.level[v-1] + 1

In [ ]:
bfs = BFS(graph, 1)

In [ ]:
bfs.search()

In [ ]:
pd.DataFrame(list(zip(range(1, bfs.graph.n_nodes+1), bfs.level, bfs.parent)), columns=["node", "level", "parent"], index=np.arange(1, bfs.graph.n_nodes+1))

In [ ]:
net.show("graph.html")

In [ ]:
def write_parent_level_txt(filename, graph):
    with open(filename, "w") as f:
        for node in graph.nodes:
            f.write(f"Node: {node.value} -- Parent: {node.parent} -- Level: {node.level}\n")

In [ ]:
filename = "outputs/bfs_tree.txt"
write_parent_level_txt(filename, bfs.graph)

In [ ]:
class MinimumPath:
    
    def __init__(self, graph):
        self.graph = graph
        self.matrix = np.full((graph.n_nodes, graph.n_nodes), fill_value=-1, dtype="int32")
        self.run()
    
    def run(self):
        for v in tqdm(range(1, self.graph.n_nodes+1)):
            bfs = BFS(self.graph, v)
            bfs.search()
            for bfs_node_index in np.argwhere(bfs.visited == 1).reshape(-1):
                self.matrix[v-1, bfs_node_index] = bfs.level[bfs_node_index]
            del bfs
    
    def get_distance(self, u, v):
        return self.matrix[u-1, v-1]
    
    def get_diameter(self):
        return np.max(self.matrix)
    
    def get_matrix(self):
        return self.matrix
    
    def get_matrix_beautiful(self):
        return pd.DataFrame(self.matrix, columns=np.arange(1, self.graph.n_nodes+1), index=np.arange(1, self.graph.n_nodes+1))
    

In [ ]:
minpath = MinimumPath(graph)

In [ ]:
minpath.get_matrix()

In [ ]:
minpath.get_distance(1, 3)

In [ ]:
minpath.get_diameter()

In [9]:
class Components:
    
    def __init__(self, graph):
        self.graph = graph
        self.visited = np.zeros(graph.n_nodes, dtype="uint8")
        self.components = []
        
        while np.argwhere(self.visited == 0).reshape(-1).shape[0] > 0:
            root = np.argwhere(self.visited == 0).reshape(-1)[0] + 1

            bfs = BFS(self.graph, root)
            bfs.search()
            
            bfs_visited_index = np.argwhere(bfs.visited == 1).reshape(-1)
            
            self.visited[bfs_visited_index] = 1
            self.components.append((bfs_visited_index+1).tolist())

    
    
    

In [10]:
components = Components(graph)

[1 1 0 ... 0 1 1]
[1 1 1 ... 0 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]


In [11]:
a = sorted(components.components, key=lambda x: len(x), reverse=True)

In [12]:
b = [len(x) for x in a]

In [13]:
c = list(zip(b, a))

In [ ]:
c

In [ ]:
from pyvis.network import Network

net = Network(notebook=True)

for v in range(1, graph.n_nodes+1):
    net.add_node(v, label=v)
    
for v, neighbors in enumerate(graph.nodes):
    for w in neighbors:
        net.add_edge(v+1, w)

net.show("graph.html")